In [76]:
import pandas as pd
from predict import predict_SRL, predict_SRL_BERT, predict_srl_group9
sentence = "I am a student"

In [79]:
import re

def parse_string(input_string):
    # define a regular expression pattern to match the string format
    pattern = r"\[(\w+): ([^]]+)\]"
    
    # search for matches using the pattern
    matches = re.findall(pattern, input_string)
    
    # convert the matches into a dictionary
    output_dict = {}
    for match in matches:
        key = match[0]
        value = match[1]
        output_dict[key] = value
    
    return output_dict

def classify(predict, gold):

    predicted_verbs =  [verb['verb'] for verb in predict['verbs']]

    predicate_gold = gold['V']

    if predicate_gold not in predicted_verbs:
        return 0
    
    else:
        verbs = predict['verbs']
        for verb in verbs:
            if verb['verb'] == predicate_gold:
                description = parse_string(verb['description'])
                


                # Check if gold matches description
                for key in description:
                    # print(description, key, gold)
                    if key not in gold.keys():
                        return 0
                    if description[key].lower() != gold[key].lower():
                        # print(description[key], gold[key])
                        return 0
        
        return 1


In [82]:
def test_one():
    # TODO check if predicted is correct
    df = pd.read_json("testdata/test1.json")
    tests = df['tests']

    for test in tests[:4]:
        original = test['proposition']
        realizations = test['realizations']
        for real in realizations:
            sentence = real['realization']
            gold_label = real['propbank_golden_labels']

            output_srl = predict_SRL(sentence)
            output_srl_bert = predict_SRL_BERT(sentence)
            output_group_9 = predict_srl_group9(sentence)


            # print(classify(output_srl, gold_label))
            print("sentence", sentence)
            print("")
            print("Gold label", gold_label)
            print("")
            print("own", output_group_9)
            print("")
            print("srl", output_srl)
            print("")
            print("srl", output_srl_bert) 
            print("")   

test_one()

sentence The cat is pursuing the dog.

Gold label {'ARG0': 'The cat', 'V': 'pursuing', 'ARG1': 'the dog'}

own (['_', 'ARG0', '_', '_', '_', '_', '_', 'ARG1', '_'], ['The', 'cat', 'is', 'pursuing', 'the', 'dog', '.'])

srl {'verbs': [{'verb': 'is', 'description': 'The cat [V: is] pursuing the dog .', 'tags': ['O', 'O', 'B-V', 'O', 'O', 'O', 'O']}, {'verb': 'pursuing', 'description': '[ARG0: The cat] is [V: pursuing] [ARG1: the dog] .', 'tags': ['B-ARG0', 'I-ARG0', 'O', 'B-V', 'B-ARG1', 'I-ARG1', 'O']}], 'words': ['The', 'cat', 'is', 'pursuing', 'the', 'dog', '.']}

srl {'verbs': [{'verb': 'is', 'description': 'The cat [V: is] pursuing the dog .', 'tags': ['O', 'O', 'B-V', 'O', 'O', 'O', 'O']}, {'verb': 'pursuing', 'description': '[ARG0: The cat] is [V: pursuing] [ARG1: the dog] .', 'tags': ['B-ARG0', 'I-ARG0', 'O', 'B-V', 'B-ARG1', 'I-ARG1', 'O']}], 'words': ['The', 'cat', 'is', 'pursuing', 'the', 'dog', '.']}
sentence The dog is being chased by the cat.

Gold label {'ARG1': 'The dog',

In [78]:
def test_two():
    # TODO check if predicted is correct
    df = pd.read_json("testdata/test2.json")
    tests = df['tests']

    for test in tests:

        statement = test['statement']
        question = test['question']

        golden_labels = test['propbank_golden_labels'][0]

        print(golden_labels)
        statement_gold = golden_labels['Statement']
        question_gold = golden_labels['Question']

        output_statement_srl = predict_SRL(statement)
        output_statement_srl_bert = predict_SRL_BERT(statement)
        output_statement_group_9 = predict_srl_group9(statement)

        output_question_srl = predict_SRL(question)
        output_question_srl_bert = predict_SRL_BERT(question)
        output_question_group_9 = predict_srl_group9(question)

        print(output_statement_group_9)

In [71]:
import re

def parse_string(input_string):
    # define a regular expression pattern to match the string format
    pattern = r"\[(\w+): ([^]]+)\]"
    
    # search for matches using the pattern
    matches = re.findall(pattern, input_string)
    
    # convert the matches into a dictionary
    output_dict = {}
    for match in matches:
        key = match[0]
        value = match[1]
        output_dict[key] = value
    
    return output_dict

def classify(predict, gold):

    predicted_verbs =  [verb['verb'] for verb in predict['verbs']]

    predicate_gold = gold['V']

    if predicate_gold not in predicted_verbs:
        return 0
    
    else:
        verbs = predict['verbs']
        for verb in verbs:
            if verb['verb'] == predicate_gold:
                description = parse_string(verb['description'])
                


                # Check if gold matches description
                for key in description:
                    # print(description, key, gold)
                    if key not in gold.keys():
                        return 0
                    if description[key].lower() != gold[key].lower():
                        # print(description[key], gold[key])
                        return 0
        
        return 1


In [72]:
def test_three():
    df = pd.read_json("testdata/test3.json")
    tests = df['tests']
    srl, bert, group9 = 0, 0, 0
    for test in tests:
        active = test['active']
        passive = test['passive']

        golden_labels = test['propbank_golden_labels'][0]

        active_gold = golden_labels['Active']
        passive_gold = golden_labels['Passive']

        output_active_srl = predict_SRL(active)
        output_active_srl_bert = predict_SRL_BERT(active)
        output_active_group_9 = predict_srl_group9(active)


        output_passive_srl = predict_SRL(passive)
        output_passive_srl_bert = predict_SRL_BERT(passive)
        output_passive_group_9 = predict_srl_group9(passive)


        # Get all verbs
        verbs = [verb['verb'] for verb in output_passive_srl['verbs']]

        # print(output_active_srl_bert,  "\n\n", active_gold)
        srl += (classify(output_active_srl, active_gold) * classify(output_passive_srl, passive_gold))
        bert += (classify(output_active_srl_bert, active_gold) * classify(output_passive_srl_bert, passive_gold))

    print(srl, bert)
test_three()

{'ARG0': 'The chef', 'V': 'cooked', 'ARG1': 'the meal'} ARG0 {'V': 'cooked', 'ARG0': 'the chef', 'ARG1': 'the meal'}
{'ARG0': 'The chef', 'V': 'cooked', 'ARG1': 'the meal'} V {'V': 'cooked', 'ARG0': 'the chef', 'ARG1': 'the meal'}
{'ARG0': 'The chef', 'V': 'cooked', 'ARG1': 'the meal'} ARG1 {'V': 'cooked', 'ARG0': 'the chef', 'ARG1': 'the meal'}
{'ARG1': 'The meal', 'V': 'cooked', 'ARG0': 'by the chef'} ARG1 {'V': 'cooked', 'ARG1': 'the meal', 'ARG0': 'by the chef'}
{'ARG1': 'The meal', 'V': 'cooked', 'ARG0': 'by the chef'} V {'V': 'cooked', 'ARG1': 'the meal', 'ARG0': 'by the chef'}
{'ARG1': 'The meal', 'V': 'cooked', 'ARG0': 'by the chef'} ARG0 {'V': 'cooked', 'ARG1': 'the meal', 'ARG0': 'by the chef'}
{'ARG0': 'The chef', 'V': 'cooked', 'ARG1': 'the meal'} ARG0 {'V': 'cooked', 'ARG0': 'the chef', 'ARG1': 'the meal'}
{'ARG0': 'The chef', 'V': 'cooked', 'ARG1': 'the meal'} V {'V': 'cooked', 'ARG0': 'the chef', 'ARG1': 'the meal'}
{'ARG0': 'The chef', 'V': 'cooked', 'ARG1': 'the meal'}

In [75]:

df = pd.read_json("testdata/test4.json")
tests = df['tests']

for test in tests:
    sentence = test['sentence']
    golden_labels = test['propbank_golden_labels']

    output_srl = predict_SRL(sentence)
    output_srl_bert = predict_SRL_BERT(sentence)
    output_group_9 = predict_srl_group9(sentence)

    print(golden_labels, "\n\n", output_srl_bert)
    print(classify(output_srl_bert, golden_labels))

{'V': 'give', 'ARG0': 'the teacher', 'ARG1': 'the students', 'ARG2': 'a quiz', 'ARGM_TMP': 'yesterday'} 

 {'verbs': [{'verb': 'was', 'description': 'It [V: was] [ARG2: the teacher] [ARG1: who gave the students a quiz yesterday] .', 'tags': ['O', 'B-V', 'B-ARG2', 'I-ARG2', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'I-ARG1', 'O']}, {'verb': 'gave', 'description': 'It was [ARG0: the teacher] [R-ARG0: who] [V: gave] [ARG2: the students] [ARG1: a quiz] [ARGM-TMP: yesterday] .', 'tags': ['O', 'O', 'B-ARG0', 'I-ARG0', 'B-R-ARG0', 'B-V', 'B-ARG2', 'I-ARG2', 'B-ARG1', 'I-ARG1', 'B-ARGM-TMP', 'O']}], 'words': ['It', 'was', 'the', 'teacher', 'who', 'gave', 'the', 'students', 'a', 'quiz', 'yesterday', '.']}
0
{'V': 'catch', 'ARG0': 'the cat', 'ARG1': 'the mouse', 'ARGM_TMP': 'last night'} 

 {'verbs': [{'verb': 'was', 'description': 'It [V: was] [ARG2: the cat] [ARG1: that caught the mouse last night] .', 'tags': ['O', 'B-V', 'B-ARG2', 'I-ARG2', 'B-ARG1', 'I-ARG1', 'I-ARG1', 'I-

In [11]:

df = pd.read_json("testdata/test5.json")
tests = df['tests']

for test in tests:
    print(test)

{'sentence': 'The teacher assigned the students a quiz.', 'propbank_golden_labels': {'V': 'assign', 'ARG0': 'the teacher', 'ARG1': 'the students', 'ARG2': 'a quiz'}}
{'sentence': 'The students were assigned a quiz by the teacher.', 'propbank_golden_labels': {'V': 'assign', 'ARG0': 'the teacher', 'ARG1': 'the students', 'ARG2': 'a quiz'}}
{'sentence': 'The burglar broke into the house.', 'propbank_golden_labels': {'V': 'break', 'ARG0': 'the burglar', 'ARG1': 'into the house'}}
{'sentence': 'The house was broken into by the burglar.', 'propbank_golden_labels': {'V': 'break', 'ARG0': 'the burglar', 'ARG1': 'into the house'}}
{'sentence': 'The chef chopped the vegetables finely.', 'propbank_golden_labels': {'V': 'chop', 'ARG0': 'the chef', 'ARG1': 'the vegetables', 'ARGM_MNR': 'finely'}}
{'sentence': 'The vegetables were chopped finely by the chef.', 'propbank_golden_labels': {'V': 'chop', 'ARG0': 'the chef', 'ARG1': 'the vegetables', 'ARGM_MNR': 'finely'}}
{'sentence': 'The baby drank the